<a href="https://colab.research.google.com/github/emilyzfliu/vis-sounds/blob/main/vis_sounds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Project Overview:

1. Sounds from Images, Images from Sounds
  * We will start with the ImageNet neural net architecture for training our model to predict sounds from images as in (https://arxiv.org/pdf/1512.08512.pdf).
  * Then, to predict images from sounds, we will reverse this architecture.
  * We will likely use open-source pretrained weights and replace the classification layers with sound identification layers (TODO: Find ImageNet pretrained weights).
  * We additionally hope to attempt an image-sound-image/sound-image-sound reconstruction by consecutively passing through the two models.
2. Generated image/sound as input in downstream tasks: can we augment visual datasets by generating new images from sounds?
  * raw images + network-generated images (from sounds) on ImageNet tasks
  * TODO: raw sounds + network-generated sounds from images (eliminate if there are no good datasets to work with here)
  * network-generated sounds + raw images in sound-supervised statistical summary task (https://arxiv.org/pdf/1608.07017.pdf)

## Datasets
We use the following datasets:
1. Owens: https://andrewowens.com/vis/hits.html
2. VoxCeleb 1, 2: https://www.robots.ox.ac.uk/~vgg/data/voxceleb/
3. VGG Sound: https://www.robots.ox.ac.uk/~vgg/data/vggsound/